# Getting started with Hugging Face and Amazon Sagemaker

## Sentiment analysis on product reviews

* https://huggingface.co/distilbert-base-uncased
* https://huggingface.co/datasets/generated_reviews_enth

{'correct': 0, 'review_star': 4, 'translation': {'en': "I had a hard time finding a case for my new LG Lucid 2 but finally found this one on amazon. The colors are really pretty and it works just as well as, if not better than the otterbox. Hopefully there will be more available by next Xmas season. Overall, very cute case. I love cheetah's. :)", 'th': 'ฉันมีปัญหาในการหาเคสสำหรับ LG Lucid 2 ใหม่ของฉัน แต่ในที่สุดก็พบเคสนี้ใน Amazon สีสวยมากและใช้งานได้ดีเช่นเดียวกับถ้าไม่ดีกว่านาก หวังว่าจะมีให้มากขึ้นในช่วงเทศกาลคริสต์มาสหน้า โดยรวมแล้วน่ารักมาก ๆ ฉันรักเสือชีตาห์ :)'}}

{'correct': 0, 'review_star': 1, 'translation': {'en': "This is the second battery charger I bought as a Christmas present, that came from Amazon, after one purchased before for my son. His was still working. The first charger, received in July, broke apart and wouldn't charge anymore. Just found out two days ago they discontinued it without warning. It took quite some time to find the exact replacement charger. Too bad, really liked it. One of these days, will purchase an actual Nikon product, or go back to buying batteries.", 'th': 'นี่เป็นเครื่องชาร์จแบตเตอรี่ก้อนที่สองที่ฉันซื้อเป็นของขวัญคริสต์มาสซึ่งมาจากอเมซอนหลังจากที่ซื้อมาเพื่อลูกชายของฉัน เขายังทำงานอยู่ เครื่องชาร์จแรกที่ได้รับในเดือนกรกฎาคมแตกเป็นชิ้น ๆ และจะไม่ชาร์จอีกต่อไป เพิ่งค้นพบเมื่อสองวันก่อนพวกเขาหยุดมันโดยไม่มีการเตือนล่วงหน้า ใช้เวลาพอสมควรในการหาที่ชาร์จที่ถูกต้อง แย่มากชอบมาก สักวันหนึ่งจะซื้อผลิตภัณฑ์ Nikon จริงหรือกลับไปซื้อแบตเตอรี่'}}

{'correct': 1, 'review_star': 1, 'translation': {'en': 'I loved the idea of having a portable computer to share pictures with family and friends on my big screen. It worked really well for about 3 days, then when i opened it one evening there was water inside where all the wires came out. I cleaned that up and put some tape over that, so far, no leaks. My husband just told me yesterday, however, that this thing is trash.', 'th': 'ฉันชอบไอเดียที่มีคอมพิวเตอร์พกพาเพื่อแชร์รูปภาพกับครอบครัวและเพื่อน ๆ บนหน้าจอขนาดใหญ่ของฉัน มันใช้งานได้ดีจริง ๆ ประมาณ 3 วันจากนั้นเมื่อฉันเปิดมันในเย็นวันหนึ่งมีน้ำอยู่ภายในที่ซึ่งสายไฟทั้งหมดออกมา ฉันทำความสะอาดมันแล้ววางเทปไว้ที่นั่นจนถึงตอนนี้ไม่มีรอยรั่ว สามีของฉันเพิ่งบอกฉันเมื่อวานนี้ว่าสิ่งนี้เป็นขยะ'}}


# Setup

In [ ]:
%%sh
pip -q install pip --upgrade
pip -q install sagemaker --upgrade

In [2]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

2.46.0


# Use processed data in S3

In [3]:
train_input_path = 's3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-16-07-57-40-837/output/training'
valid_input_path = 's3://sagemaker-eu-west-1-613904931467/sagemaker-scikit-learn-2021-06-16-07-57-40-837/output/validation'

# Fine-tuning & starting Sagemaker Training Job

In [4]:
!pygmentize train.py

import random, sys, argparse, os, logging, torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_from_disk

if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--train-batch-size", type=int, default=32)
    parser.add_argument("--eval-batch-size", type=int, default=64)
    parser.add_argument("--save-strategy", type=str, default='no')
    parser.add_argument("--save-steps", type=int, default=500)
    parser.add_argument("--model-name", type=str)
    parser.add_argument("--learning-rate", type=str, default=5e-5)

    # Data, model, and output directories
    parser.add_argument("--output-data-dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser

## Fine-tune the Hugging Face model on SageMaker

In [5]:
hyperparameters={
    'epochs': 1,
    'train-batch_size': 32,
    'model-name':'distilbert-base-uncased'
}

In [6]:
from sagemaker.huggingface import HuggingFace

huggingface_estimator = HuggingFace(
    role=sagemaker.get_execution_role(),
    # Fine-tuning script
    entry_point='train.py',
    hyperparameters=hyperparameters,
    # Infrastructure
    transformers_version='4.4.2',
    pytorch_version='1.6.0',
    py_version='py36',
    instance_type='ml.p3.8xlarge',
    instance_count=1
)

In [7]:
huggingface_estimator.fit({'train': train_input_path, 'valid': valid_input_path})

2021-06-16 08:09:28 Starting - Starting the training job...
2021-06-16 08:09:51 Starting - Launching requested ML instancesProfilerReport-1623830968: InProgress
.........
2021-06-16 08:11:12 Starting - Preparing the instances for training.........
2021-06-16 08:12:53 Downloading - Downloading input data...
2021-06-16 08:13:26 Training - Downloading the training image............
2021-06-16 08:15:19 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-06-16 08:15:20,589 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-06-16 08:15:20,631 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-06-16 08:15:23,679 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-06-16 08:15:24,156 sagemaker-training-toolkit INFO     Invoking user script

Training

In [8]:
huggingface_estimator.model_data

's3://sagemaker-eu-west-1-613904931467/huggingface-pytorch-training-2021-06-16-08-09-27-826/output/model.tar.gz'

# Retrieve model

In [9]:
%%sh -s $huggingface_estimator.model_data
aws s3 cp $1 .
mkdir -p model
tar -xvzf model.tar.gz -C model

download: s3://sagemaker-eu-west-1-613904931467/huggingface-pytorch-training-2021-06-16-08-09-27-826/output/model.tar.gz to ./model.tar.gz
training_args.bin
config.json
pytorch_model.bin


## Deploy the Hugging Face model with PyTorch on SageMaker

In [19]:
!pygmentize src/torchserve-predictor.py

import json
import torch
from transformers import AutoConfig, AutoTokenizer, DistilBertForSequenceClassification

JSON_CONTENT_TYPE = 'application/json'
CLASS_NAMES = ['negative', 'positive']

def model_fn(model_dir):
    config_path = '{}/config.json'.format(model_dir)
    model_path =  '{}/pytorch_model.bin'.format(model_dir)
    config = AutoConfig.from_pretrained(config_path)
    model = DistilBertForSequenceClassification.from_pretrained(model_path, config=config)
    print(config)
    return model

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def predict_fn(input_data, model):
    inputs = tokenizer(input_data['text'], return_tensors='pt')
    outputs = model(**inputs)
    logits = outputs.logits
    _, prediction = torch.max(logits, dim=1)
    return CLASS_NAMES[prediction]

def input_fn(serialized_input_data, content_type=JSON_CONTENT_TYPE):  
    if content_type == JSON_CONTENT_TYPE:
        input_data = json.loads(serialized_input_data)
        return

In [20]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    model_data=huggingface_estimator.model_data,
    role=sagemaker.get_execution_role(), 
    entry_point='torchserve-predictor.py',
    source_dir='src',
    framework_version='1.6.0',
    py_version='py36')

In [21]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: pytorch-inference-2021-06-16-08-33-12-939
INFO:sagemaker:Creating endpoint with name pytorch-inference-2021-06-16-08-33-13-259


-------------!

In [22]:
test_data = {'text': "This is a very nice camera, I'm super happy with it."}

In [25]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

prediction = predictor.predict(test_data)

print(prediction)

positive


In [26]:
test_data = {'text': "Terrible purchase, I want my money back!"}

In [27]:
prediction = predictor.predict(test_data)
print(prediction)

negative


In [ ]:
#predictor.delete_endpoint()